In [25]:
from kiteconnect import KiteConnect
import logging 
import os 
import datetime as dt 
import pandas as pd 

In [26]:
curr_dir = os.chdir(r'C:\Users\rsurs\OneDrive\Documents\Zerodha\api_keys')
#print(os.getcwd())

# Generate a trading session

In [27]:
request_token = open('request_token.txt','r').read()
key_secret = open('credentials.txt','r').read().split()
access_token = open('access_token.txt','r').read()
api_key = key_secret[0]
api_secret = key_secret[1]
kite = KiteConnect(api_key=api_key)
kite.set_access_token(access_token=access_token)

In [28]:
instruments_dump = kite.instruments('NSE')
instrument_df = pd.DataFrame(instruments_dump) 
instrument_df.to_csv('NSE_instruments.csv',index=False)

In [29]:
instrument_df.head()

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,256265,1001,NIFTY 50,NIFTY 50,0.0,,0.0,0.0,0,EQ,INDICES,NSE
1,256777,1003,NIFTY MIDCAP 100,NIFTY MIDCAP 100,0.0,,0.0,0.0,0,EQ,INDICES,NSE
2,260105,1016,NIFTY BANK,NIFTY BANK,0.0,,0.0,0.0,0,EQ,INDICES,NSE
3,260617,1018,NIFTY 100,NIFTY 100,0.0,,0.0,0.0,0,EQ,INDICES,NSE
4,257033,1004,NIFTY DIV OPPS 50,NIFTY DIV OPPS 50,0.0,,0.0,0.0,0,EQ,INDICES,NSE


In [30]:

def instrument_token(instrument_df, symbol):
    """
    This function will return the token number of the instrument from data
    """
    return instrument_df[instrument_df.tradingsymbol == symbol].instrument_token.values[0]

In [31]:
instrument_token(instrument_df,'RELIANCE')

738561

# Retrieve historic data from an instrument

In [34]:
def fetchOHLC(ticker,interval,duration):
    instrument = instrument_token(instrument_df,ticker)
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration), dt.date.today(),interval))
    data.set_index("date",inplace=True)
    return data

In [42]:
df = fetchOHLC('INFY','5minute',5)

In [43]:
df

,open,high,low,close,volume
date,,,,,
2024-05-21 09:15:00+05:30,1430.00,1444.70,1429.70,1436.30,354639
2024-05-21 09:20:00+05:30,1436.35,1438.45,1435.25,1436.10,133221
2024-05-21 09:25:00+05:30,1436.10,1436.10,1434.00,1434.25,94873
2024-05-21 09:30:00+05:30,1434.25,1435.00,1433.70,1434.00,101265
2024-05-21 09:35:00+05:30,1434.00,1434.20,1430.85,1431.70,170131
...,...,...,...,...,...
2024-05-24 15:05:00+05:30,1465.80,1468.55,1465.50,1468.05,168201
2024-05-24 15:10:00+05:30,1468.30,1468.85,1466.00,1466.15,152465
2024-05-24 15:15:00+05:30,1466.15,1468.00,1464.35,1464.45,180447


The limit for per minute data is 60 days. Similarly for other time intervals there are limits as stated below;

- Minute: 60 days
- 3 minutes: 100 days
- 5 minutes: 100 days
- 10 minutes: 100 days
- 15 minutes: 200 days
- 30 minutes: 200 days
- 60 minutes: 400 days
- 1 Day: 2000 days


# Overcoming the time limits

In [50]:
def historical_data(symbol, from_date, to_date, interval):
    df = pd.DataFrame()
    inst_token = instrument_token(instrument_df, symbol)
    to_date = pd.Timestamp(to_date)
    from_date = pd.Timestamp(from_date)
    
    while True:
        if from_date >= (to_date - dt.timedelta(60)):
            new_df = pd.concat([df, pd.DataFrame(kite.historical_data(inst_token, from_date, to_date, interval))])
            break
        else:
            to_date_new = from_date + dt.timedelta(60)
            new_df = pd.concat([df, pd.DataFrame(kite.historical_data(inst_token, from_date, to_date_new, interval))])
            from_date = to_date_new
    return new_df

In [51]:
NIFTY_50 = historical_data('NIFTY 50', '01-01-2020', '31-12-2023', 'minute')
NIFTY_50

,date,open,high,low,close,volume
0,2023-12-11 09:15:00+05:30,20965.30,20988.15,20923.70,20977.60,0
1,2023-12-11 09:16:00+05:30,20977.85,20980.45,20969.90,20976.75,0
2,2023-12-11 09:17:00+05:30,20976.80,20989.60,20975.15,20989.25,0
3,2023-12-11 09:18:00+05:30,20991.05,20999.40,20985.70,20998.20,0
4,2023-12-11 09:19:00+05:30,20997.15,21004.95,20991.65,21004.65,0
...,...,...,...,...,...,...
5245,2023-12-29 15:25:00+05:30,21723.35,21729.15,21722.50,21727.90,0
5246,2023-12-29 15:26:00+05:30,21726.00,21731.45,21725.10,21730.90,0
5247,2023-12-29 15:27:00+05:30,21731.20,21738.40,21729.35,21736.30,0
5248,2023-12-29 15:28:00+05:30,21736.40,21739.35,21729.65,21734.65,0
